# Training

In [1]:
from config.config import *
import tf_keras as keras
import tensorflow as tf
import pandas as pd
import numpy as np

2024-12-13 13:13:05.980110: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-13 13:13:06.331726: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1734075786.473273  163406 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1734075786.520339  163406 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-13 13:13:06.893805: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
from processing.preprocessing import Dataset
from training.model import create_model, plot_model

In [3]:
train_dataset = Dataset.create_training_pipeline(DF)
val_dataset = Dataset.create_validation_pipeline(DF)

2024-12-13 09:22:48.875563: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2024-12-13 09:22:48.875611: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[None_lookup_table_find/LookupTableFindV2/_4]]
2024-12-13 09:22:48.875629: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 15068800475560787303
2024-12-13 09:22:48.875651: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 1143512306394923478
2024-12-13 09:22:48.875657: I tensorflow/core/framework/local_rendezvous.cc:424] Local rendezvous recv item cancelled. Key hash: 8693832692366513368
2024-12-13 09:22:59.158299: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANG

In [4]:
keras.mixed_precision.set_global_policy('mixed_float16')

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4050 Laptop GPU, compute capability 8.9


In [3]:
model = create_model()

I0000 00:00:1734075801.271335  163406 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3539 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


In [ ]:
history = model.fit(
    train_dataset.pipeline.prefetch(tf.data.AUTOTUNE),
    validation_data=val_dataset.pipeline.prefetch(tf.data.AUTOTUNE),
    initial_epoch=0,
    epochs=3,
    shuffle=False,
    batch_size= BATCH_SIZE,
    callbacks=[
        keras.callbacks.ModelCheckpoint(
            filepath=CHECKPOINTS_DIR.joinpath(f'{model.name}.weights.h5'), 
            save_best_only=True, 
            save_weights_only=True, 
            monitor='val_accuracy', 
            verbose=1
        )
    ],
)

Epoch 1/3
562/562 [==============================] - ETA: 0s - loss: 1.1373 - accuracy: 0.6819
Epoch 1: val_accuracy improved from -inf to 0.81117, saving model to /home/yehand/.vscode-server/projects/NLP_Learning/RCTabstractmodel/training/checkpoints/model_2.weights.h5
562/562 [==============================] - 72s 116ms/step - loss: 1.1373 - accuracy: 0.6819 - val_loss: 0.9649 - val_accuracy: 0.8112
Epoch 2/3
562/562 [==============================] - ETA: 0s - loss: 0.9061 - accuracy: 0.8549
Epoch 2: val_accuracy did not improve from 0.81117
562/562 [==============================] - 40s 72ms/step - loss: 0.9061 - accuracy: 0.8549 - val_loss: 0.9585 - val_accuracy: 0.8072
Epoch 3/3
562/562 [==============================] - ETA: 0s - loss: 0.8341 - accuracy: 0.9084
Epoch 3: val_accuracy did not improve from 0.81117
562/562 [==============================] - 41s 73ms/step - loss: 0.8341 - accuracy: 0.9084 - val_loss: 0.9854 - val_accuracy: 0.7955


In [13]:
model.save(SERIALIZATION_DIR.joinpath('model_test.keras'))

In [ ]:
name = 'model_test'

test_dataset = Dataset.create_test_pipeline(DF)
keras.mixed_precision.set_global_policy('mixed_float16')

lmodel = keras.models.load_model(SERIALIZATION_DIR.joinpath(f'{name}.keras'))

lpredictions = model.predict(test_dataset)